In [108]:
import sqlite3
import json
from datetime import datetime

In [109]:
timeframe = '2005-12'
sql_transaction = []

In [110]:
connection = sqlite3.connect('{}.db'.format(timeframe))

In [111]:
c = connection.cursor()

In [112]:
def create_table():
    c.execute("""CREATE TABLE IF NOT EXISTS parent_reply(parent_id TEXT PRIMARY KEY, comment_id TEXT, parent TEXT, comment TEXT, subreddit TEXT, unix INT, score INT)""")
    connection.commit()

In [113]:
def format_data(data):
    data = data.replace("\n", " newlinechar ").replace("\r", " newlinechar ").replace('"', "''")
    return data

In [114]:
def find_parent(pid):
    try:
        sql = "SELECT comment FROM parent_reply WHERE comment_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        print(f'{result}')
        if result != None:
            return result[0]
        else:
            return False
            connection.commit()
    except Exception as e:
        return False

    

In [115]:
def find_existing_score(pid):
    try:
        sql = "SELECT score FROM parent_reply WHERE parent_id = '{}' LIMIT 1".format(pid)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else:
            return False
        connection.commit()
    except Exception as e:
        return False

    

In [116]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else:
        return True

In [117]:
def transaction_bldr(sql):
    global sql_transaction
    sql_transaction.append(sql)
    if len(sql_transaction) > 1000:
        c.execute('BEGIN TRANSACTION')
        for s in sql_transaction:
            try:
                c.execute(s)
            except:
                pass
        connection.commit()
        sql_transaction = []

In [118]:
def sql_insert_replace_comment(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """UPDATE parent_reply SET parent_id = ?, comment_id = ?, comment, subreddit = ?, unix = ?, score = ? WHERE parent_id = ?""".format(parentid)
        transaction_bldr(sql)
    except Exception as e:
        print('s-UPDATE insertion', str(e))
        
def sql_insert_has_parent(commentid, parentid, parent, comment, subreddit, time, score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, parent, comment, subreddit, time, score) VALUES("{}","{}","{}","{}","{}","{}","{}");""".format(parentid, commentid, parent, comment, subreddit, time, score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-INSERT-has-parent', str(e))
        
def sql_insert_no_parent(commentid, parentid, comment, subreddit, time, score):
    try:
        sql = """INSERT INTO parent_reply (parent_id, comment_id, subreddit, time, score)VALUES("{}","{}","{}","{}","{}","{}");""".format(parentid, commentid, comment, subreddit, time, score)
        transaction_bldr(sql)
    except Exception as e:
        print('s-INSERT-no-parent', str(e))

In [119]:
create_table()
row_counter = 0
paired_rows = 0

#     with open("C:\\Users\\adity\\OneDrive\\Desktop\\NLP\\RC_{}".format(timeframe.split('-')[0], timeframe)) as f:
with open("C:\\Users\\adity\\OneDrive\\Desktop\\NLP\\RC_2005-12\\RC_2005-12") as f:
      for row in f:
#                 print(row)
            row_counter += 1
            row = json.loads(row)
            parent_id = row['parent_id']
            body = format_data(row['body'])
            created_utc = row['created_utc']
            score = row['score']
            subreddit = row['subreddit']
            comment_id = row['subreddit_id']
            parent_data = find_parent(parent_id)

            if score >= 2:
                if acceptable(body):
                    existing_comment_score = find_existing_score(parent_id)
                    if existing_comment_score:
                        if score > existing_comment_score:
                            sql_insert_replace_comment(comment_id, parent_id, parent_data,body, subreddit, created_utc, score)

                    else:
                        print(parent_data)
                        if parent_data:
                            print('here')
                            sql_insert_has_parent(comment_id, parent_id, parent_data,body, subreddit, created_utc, score)
                            paired_rows += 1
                        else:
                            sql_insert_no_parent(comment_id, parent_id, body, subreddit, created_utc, score)


            if row_counter % 1000 == 0:
                print("total rows read:{}, Paired rows:{}, Time:{}".format(row_counter, paired_rows, str(datetime.now())))

                

None
False
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
False
None
False
None
False
None
False
None
None
None
None
None
None
None
None
None
None
None
None
None
None
False
None
None
None
False
None
None
None
False
None
False
None
None
None
None
False
None
None
None
None
None
None
False
None
None
None
False
None
None
None
None
False
None
False
None
None
None
False
None
False
None
False
None
None
False
None
False
None
None
False
None
None
False
None
None
None
None
False
None
False
None
None
False
None
None
None
None
None
False
None
False
None
None
None
None
None
None
None
None
None
None
None
None
False
None
None
None
None
False
None
False
None
None
None
False
None
None
None
None
None
None
False
None
None
None
None
None
None
None
None
None
None
False
None
None
None
None
None
False
None
None
False
None
None
None
None
None
None
None
None
None
None
None
False
None
False
None
False
None
None
None
None
False
None
None
None
False
None
False
None
None
None
